In [1]:
import pandas as pd
from scipy import stats
import plotly

In [2]:
df = pd.read_csv('stats.csv')

In [3]:
df['Has_discount'] = df['Discount'].apply(lambda x: True if x > 0  else False)

In [22]:
df.groupby('Quantity')['Discount'].agg('count')

Quantity
1     25
2     25
3     25
4     26
5     25
6     27
7     22
8     22
9     22
10    18
11    14
12    17
13    11
14    12
Name: Discount, dtype: int64

In [5]:
with_discount = df[df['Has_discount'] == True]['Quantity']
without_discount = df[df['Has_discount'] == False]['Quantity']

In [18]:
import plotly.express as px

fig = px.histogram(df, x='Quantity',
                    color='Has_discount',
                    marginal='box',
                    hover_data=df.columns)
fig.show()

In [26]:
_, pval = stats.shapiro(df['Quantity'])
if pval < 0.05:
    print('The distribution of data is not normal')
else:
    print('The distribution of data is normal')


The distribution of data is not normal


In [32]:
_, pval = stats.mannwhitneyu(with_discount, without_discount)

In [36]:
if pval < 0.05:
    print('There is no significant difference between the two group. rejecting null hypothesis')
else:
    print('Fail to reject null hypothesis')

Fail to reject null hypothesis
